# Práctica 4 Reconocimiento de Formas: Clasificador Bayesiano Paramétrico

* **Alumno 1**: Javier Barragán Haro
* **Alumno 2**: Victor Nieves Sanchez


In [18]:
import numpy as np
from sklearn.covariance import ShrunkCovariance
from sklearn import preprocessing
from abc import abstractmethod
import pandas as pd


class Classifier:

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifBayesianoParametrico(Classifier):
    def __init__(self, share_covs=False, shrinkage=0.0):
        """Constructor de la clase
        share_covs: Indica si la matriz de covarianzas va a ser compartida entre las distintas clases.
        shrinkage: Parámetro que determina la diagonalidad de la matriz de covarianzas. Ver sklearn.covariance.ShrunkCovariance
        """
        assert 0 <= shrinkage <= 1
        self.labels = None
        self.ln_apriories = None
        self.means = None
        self.ln_determinants = None
        self.inv_covs = None
        self.share_covs = share_covs
        self.shrinkage = shrinkage
        self.scaler = preprocessing.StandardScaler()

    def fit(self, X, y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        assert X.ndim == 2 and X.shape[0] == len(y)
        
        self.labels = y

        # Aseguramos que las etiquetas son numeros tal que: [0, 1, ..., N]
        y = pd.factorize(y)[0]
        
        # Preprocesamos los datos de entrada
        X = self.scaler.fit_transform(X)    
        
        # Contar cuantos ejemplos hay de cada etiqueta
        unique, counts = np.unique(y, return_counts=True)
        
        self.ln_apriories = np.array([np.log((counts[i]/np.sum(counts))) for i in unique])
        
        # Calcular para los ejemplos de cada clase, la media de cada una de sus características (centroide)
        self.means = np.array([np.mean(X[y==i], axis=0) for i in unique])
        
        if self.share_covs:
            # Restamos al dato de cada clase su centroide
            new_X = np.array([])
            #for i in unique:
            new_X = [np.append(new_X.reshape(-1, X.shape[1]), X[y==i] - self.means[i], axis=0) for i in unique]

            # Calcula la matriz de covarianzas empleando la clase ShrunkCovariance de sklearn
            cov = ShrunkCovariance(shrinkage=self.shrinkage).fit(new_X).covariance_

            # La reproducimos tantas veces como número de clases
            covs = np.tile(cov, (len(unique), 1, 1))

        else:
            # Calcula la matriz de covarianzas empleando la clase ShrunkCovariance de sklearn
            covs = np.array([ShrunkCovariance(shrinkage=self.shrinkage).fit(X[y==i]).covariance_ for i in unique])
            
        # Para cada una de las clases, calcular el logaritmo neperiano de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.ln_determinants = np.log(list(map(np.linalg.det, covs)))
        # Para cada una de las clases, calcular la inversa de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.inv_covs = np.array(list(map(np.linalg.pinv,covs)))
        
        return self

    def predict(self, X):
        """Estima el grado de pertenencia de cada dato a todas las clases
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas.
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase"""
        
        assert self.means is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.means.shape[1]

        # Preprocesamos nuestros datos
        X = self.scaler.fit_transform(X)        

        # Resta la media de cada clase a cada ejemplo en X
        new_X = self.means[:,np.newaxis,:] - X 
        
        # Calcula el logaritmo de la función de decisión gausiana
        # Transparencias de Métodos paramétricos de clasificación: página 14:
        # -(1/2)ln|Sigma_i| - (1/2)*(x- mu_i)^T Sigma_i^-1 (x- mu_i) + lnP(alpha_i)
        return -0.5 * self.ln_determinants[:, np.newaxis] \
            - 0.5 * np.array([np.sum((new_X[i] @ self.inv_covs[i]) 
                                     * new_X[i], axis=1) for i in np.unique(y)]) \
            + self.ln_apriories[:, np.newaxis]
    
    def pred_label(self, X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        return np.argmax(X, axis = 0)
    
    def num_aciertos(self, X): 
        """Cuenta el numero de aciertos del clasificador para un conjunto de datos X.
        X: matriz de datos a clasificar"""
        same_values = []
        [same_values.append(X[i] == self.labels[i]) for i in range(0, len(self.labels))]
        number = same_values.count(True)

        return number, (number / len(X)) * 100  

## Iris Dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Iris:

In [31]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
# print("X: \n" + str(X))
y = dataset.target

# 2. Entrenar el clasificador
ClassifBayesP = ClassifBayesianoParametrico()
ClassifBayesP.fit(X,y)

# 3. Predecir empleando la base de datos de entrenamiento (X)
predict_matrix = ClassifBayesP.predict(X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
labels_matrix = ClassifBayesP.pred_label(predict_matrix)

correct = ClassifBayesP.num_aciertos(labels_matrix)
print("Correct answers:", correct[0], "/", len(y))
print("Success rate:", correct[1])

Correct answers: 147 / 150
Success rate: 98.0


## Wine dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Wine:

In [32]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_wine
dataset = load_wine()
X = dataset.data
# print("X: \n" + str(X))
y = dataset.target
# print("y: \n" + str(y))

# 2. Entrenar el clasificador
ClassifBayesP = ClassifBayesianoParametrico()
ClassifBayesP.fit(X,y)

# 3. Predecir empleando la base de datos de entrenamiento (X)
predict_matrix = ClassifBayesP.predict(X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
labels_matrix = ClassifBayesP.pred_label(predict_matrix)

correct = ClassifBayesP.num_aciertos(labels_matrix)
print("Correct answers:", correct[0], "/", len(y))
print("Success rate:", correct[1])

Correct answers: 177 / 178
Success rate: 99.43820224719101


## Breast cancer dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Breast cancer:

In [21]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()
X = dataset.data
# print("X: \n" + str(X))
y = dataset.target
# print("y: \n" + str(y))

# 2. Entrenar el clasificador
ClassifBayesP = ClassifBayesianoParametrico()
ClassifBayesP.fit(X,y)

# 3. Predecir empleando la base de datos de entrenamiento (X)
predict_matrix = ClassifBayesP.predict(X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
labels_matrix = ClassifBayesP.pred_label(predict_matrix)

correct = ClassifBayesP.num_aciertos(labels_matrix)
print("Correct answers:", correct[0], "/", len(y))
print("Success rate:", correct[1])

Correct answers: 555 / 569
Success rate: 97.53954305799648


## MNIST Database (Modified National Institute of Standards and Technology database)
MNIST es una base de datos de texto manuscrito, que se usa de forma clásica para entrenar sistemas de procesado de imágenes.

Carga, entrenamiento, predicción y evaluación en la base de datos de MNIST:

In [30]:
from sklearn.datasets import fetch_openml

# Cargamos MNIST desde internet ( https://www.openml.org/d/554 )
all_X, all_y = fetch_openml('mnist_784', version=1, return_X_y=True, cache=True)
X = all_X[:60000]
y = all_y[:60000]
y = pd.factorize(y)[0]

# print("X: \n" + str(X))
# print("y: \n" + str(y))

# 2. Entrenar el clasificador
ClassifBayesP = ClassifBayesianoParametrico(share_covs=True, shrinkage=0.3)
ClassifBayesP.fit(X,y)

# 3. Predecir empleando la base de datos de entrenamiento (X)
predict_matrix = ClassifBayesP.predict(X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
labels_matrix = ClassifBayesP.pred_label(predict_matrix)

correct = ClassifBayesP.num_aciertos(labels_matrix)
print("Correct answers:", correct[0], "/", len(y))
print("Success rate:", correct[1])

Correct answers: 52287 / 60000
Success rate: 87.145


## Isolet Dataset (Isolated Letter Speech Recognition)
Carga, entrenamiento, predicción y evaluación en la base de datos de Isolet

In [34]:
from sklearn.datasets import fetch_openml

# 1. Cargar los datos de la base de datos de entrenamiento
X, y = fetch_openml('isolet', version=1, return_X_y=True, cache=True)
y = pd.factorize(y)[0]
# print("X: \n" + str(X))
# print("y: \n" + str(y))

# 2. Entrenar el clasificador
ClassifBayesP = ClassifBayesianoParametrico(share_covs=True, shrinkage=0.26) 
ClassifBayesP.fit(X,y)

# 3. Predecir empleando la base de datos de entrenamiento (X)
predict_matrix = ClassifBayesP.predict(X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
labels_matrix = ClassifBayesP.pred_label(predict_matrix)

correct = ClassifBayesP.num_aciertos(labels_matrix)
print("Correct answers:", correct[0], "/", len(y))
print("Success rate:", correct[1])

Correct answers: 7489 / 7797
Success rate: 96.04976272925484


Resultados de los tres experimentos:

| Base de datos | Número de aciertos | Porcentaje de aciertos |
| --- | --- | --- |
| Iris   |  147 / 150|98.00|
| Wine   | 177 / 178|99.44 | 
| Cancer | 555 / 569|97.54 |
| MNIST  | 52287 / 60000|87.15 |
| Isolet | 7489 / 7797 |  96.05 |